In [11]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret():

    secret_name = "DBCreds"
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    # Parse the secret string to get the credentials
    secret_dict = json.loads(secret)
    username = secret_dict['username']
    password = secret_dict['password']
    host = secret_dict['host']
    port = secret_dict['port']
    dbname = secret_dict['dbname']

    return username, password, host, port, dbname


(user,pswd,host,port,db) = get_secret()

In [12]:
import psycopg2
from psycopg2 import sql

# Database details
db_details = {
    'dbname': db,
    'user': user,
    'password': pswd,
    'host': host,
    'port': port
}

# Connect to PostgreSQL
try:
    conn = psycopg2.connect(**db_details)
    cursor = conn.cursor()
    print("Connected to PostgreSQL successfully.")
except Exception as e:
    print(f"Failed to connect to PostgreSQL: {e}")
    exit()

# Create the table
try:
    create_table_query = sql.SQL("""
    CREATE TABLE IF NOT EXISTS outcomes (
        uid text PRIMARY KEY,
        outcome INTEGER NOT NULL CHECK (outcome IN (0, 1))
    )
    """)

    cursor.execute(create_table_query)
    conn.commit()
    print("Table 'outcomes' created successfully.")
except Exception as e:
    print(f"Failed to create table: {e}")
    conn.rollback()
finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()
    print("PostgreSQL connection closed.")


Connected to PostgreSQL successfully.
Table 'outcomes' created successfully.
PostgreSQL connection closed.


In [ ]:
# testing it

In [2]:
import psycopg2
import uuid

# Database details
db_details = {
    'dbname': db,
    'user': user,
    'password': pswd,
    'host': host,
    'port': port
}

# Data to insert
outcome_data = {
    'id': str(uuid.uuid4()),  # Generating a unique ID
    'outcome': 1  # This can be 0 or 1
}

# Connect to PostgreSQL
try:
    conn = psycopg2.connect(**db_details)
    cursor = conn.cursor()
    print("Connected to PostgreSQL successfully.")
except Exception as e:
    print(f"Failed to connect to PostgreSQL: {e}")
    exit()

# Insert data into the table
try:
    insert_query = """
    INSERT INTO outcomes (id, outcome)
    VALUES (%s, %s)
    """
    cursor.execute(insert_query, (outcome_data['id'], outcome_data['outcome']))
    conn.commit()
    print("Data inserted successfully.")
except Exception as e:
    print(f"Failed to insert data: {e}")
    conn.rollback()
finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()
    print("PostgreSQL connection closed.")

Connected to PostgreSQL successfully.
Data inserted successfully.
PostgreSQL connection closed.


In [9]:
import pandas as pd
import psycopg2

# Database details
db_details = {
    'dbname': db,
    'user': user,
    'password': pswd,
    'host': host,
    'port': port
}

# Connect to PostgreSQL
try:
    conn = psycopg2.connect(**db_details)
    cursor = conn.cursor()
    print("Connected to PostgreSQL successfully.")
except Exception as e:
    print(f"Failed to connect to PostgreSQL: {e}")
    exit()

# Query to fetch data from the table
try:
    fetch_query = "SELECT * FROM outcomes join malware_data on malware_data.uid=outcomes.uid"
    df = pd.read_sql(fetch_query, conn)
    
    print("Data from 'outcomes' table:")
except Exception as e:
    print(f"Failed to fetch data: {e}")
finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()
    print("PostgreSQL connection closed.")

Connected to PostgreSQL successfully.
Data from 'outcomes' table:
                                    uid  outcome                  timestamp  \
0  0181a56a-8c80-42e3-9a16-8ab4cd4bc690        1 2024-07-08 20:57:21.034798   
1  e8567e15-513d-4e8e-a975-42dba1eed214        1 2024-07-08 20:57:21.073112   
2  6cb7751b-f34c-4f92-aea4-82ebcfc1bc8c        1 2024-07-08 20:57:21.173965   
3  e77848bf-3d80-4e5b-9e37-e35d25161583        1 2024-07-08 20:57:21.212819   

                                    uid  datatype  sizeofoptionalheader  \
0  0181a56a-8c80-42e3-9a16-8ab4cd4bc690         2             -0.360507   
1  e8567e15-513d-4e8e-a975-42dba1eed214         2             -0.360507   
2  6cb7751b-f34c-4f92-aea4-82ebcfc1bc8c         2             -0.360507   
3  e77848bf-3d80-4e5b-9e37-e35d25161583         2             -0.360507   

   minorlinkerversion  sizeofcode  sizeofinitializeddata  \
0           -0.376285    0.731982              -0.218904   
1           -0.376285    0.052683         

/usr/local/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [ ]:
# how to pull out kserve
print(kserve_response['predictions'][0])